# Notebook 7 - Prime Plaquettes & Topological Mass

**Level:** Intermediate  
**Topics:** Prime sequences, plaquettes, Wilson loops, topological mass, Standard Model

> **Provenance:** This notebook is based on:
> - `examples/prime_plaquette.py`
> - `light_theory_realm/pocket_u_lite` prime utilities


In [ ]:
import math
import jax
import jax.numpy as jnp
import pandas as pd
import matplotlib.pyplot as plt

from light_theory_realm.engine import CliffordEngine
from light_theory_realm.experiments.prime_gauge.prime_plaquette import PrimeLattice, PrimeWilsonLoop
from light_theory_realm.pocket_u_lite.mass_model import physical_mass

plt.rcParams["figure.figsize"] = (7, 5)

# Helper: build plaquette info for a lepton
PRIME_MAP = {
    "e": [2, 3, 5, 7],
    "mu": [11, 13, 17, 19],
    "tau": [23, 29, 31, 37],
}

engine = CliffordEngine(seed=137)
pwl = PrimeWilsonLoop(engine)


def build_lepton_plaquette(name: str):
    primes = PRIME_MAP[name]

    # Classical edge params for first link
    lattice = PrimeLattice({i: p for i, p in enumerate(primes)})
    J, A = lattice.compute_edge_params(0, 1)

    # Quantum topological mass = phase of Wilson loop expectation
    psi = engine.random_spinor(jax.random.PRNGKey(42))
    W = pwl.measure_loop(psi, primes)
    mass_phase_rad = float(jnp.angle(W))

    # Physical mass from Pocket_U Lite
    phys = physical_mass(name)["m_phys_MeV"]

    return {
        "name": name,
        "primes": primes,
        "J": float(J),
        "A_rad": float(A),
        "m_topo_rad": mass_phase_rad,
        "m_phys_MeV": float(phys),
    }


def sweep_single_prime(name: str, prime_index: int, sweep_values):
    base = PRIME_MAP[name]
    rows = []
    for val in sweep_values:
        primes = list(base)
        primes[prime_index] = val
        lattice = PrimeLattice({i: p for i, p in enumerate(primes)})
        J, A = lattice.compute_edge_params(0, 1)
        psi = engine.random_spinor(jax.random.PRNGKey(100 + val))
        W = pwl.measure_loop(psi, primes)
        mass_phase_rad = float(jnp.angle(W))
        rows.append(
            {
                "prime_value": val,
                "J": float(J),
                "A_rad": float(A),
                "m_topo_rad": mass_phase_rad,
            }
        )
    return rows


## 1. Electron, muon, tau plaquettes

We will reconstruct the prime plaquettes for the charged leptons and extract:

- J: (discrete) action or loop index
- A: area or Wilson-loop-like quantity
- m_topo: a topological mass derived from the plaquette (for example an angle)


In [ ]:
if not PRIME_MAP:
    raise RuntimeError("Prime map missing.")

species = ["e", "mu", "tau"]

plaquette_rows = []
for name in species:
    data = build_lepton_plaquette(name)
    plaquette_rows.append(data)

df_plaquettes = pd.DataFrame(plaquette_rows)
df_plaquettes


## 2. Topological mass vs physical mass

First plot: physical mass (MeV) vs topological mass phase (rad) on a shared log axis (mixed units).
The sweep plot below zooms in on the topological phase response in radians.


In [ ]:
plt.figure()
x = range(len(df_plaquettes))

plt.bar(x, df_plaquettes["m_phys_MeV"], alpha=0.5, label="Physical mass [MeV]")
plt.bar(x, df_plaquettes["m_topo_rad"], alpha=0.5, label="Topological mass [rad]")

plt.xticks(x, df_plaquettes["name"])
plt.yscale("log")
plt.ylabel("Mass scale (mixed units, log)")
plt.title("Physical vs topological mass from prime plaquettes")
plt.legend()
plt.tight_layout()
plt.show()


## 3. Sweeping one prime in a plaquette

To see how topological mass responds to arithmetic changes, we will:

- Fix all but one prime in the plaquette.
- Vary that prime over a small range.
- Plot the resulting topological mass.


In [ ]:
target = "e"      # for example, electron plaquette
prime_idx = 0     # which prime in the sequence to vary
sweep_range = range(11, 60)  # example small range of primes or integers

sweep_data = sweep_single_prime(
    name=target,
    prime_index=prime_idx,
    sweep_values=list(sweep_range),
)

df_sweep = pd.DataFrame(sweep_data)
df_sweep.head()


In [ ]:
plt.figure()
plt.plot(df_sweep["prime_value"], df_sweep["m_topo_rad"])
plt.xlabel(f"Prime at index {prime_idx}")
plt.ylabel("Topological mass m_topo [MeV]")
plt.title(f"Effect of varying a single prime on {target} topological mass")
plt.grid(True, linestyle="--", alpha=0.4)
plt.tight_layout()
plt.show()


## 4. Interpretation: primes to masses

Things to observe:
- How sensitive is m_topo to changes in a single prime?
- Do some prime positions matter more than others?
- How do e, mu, and tau differ in their prime patterns?

Next steps:
- Try sweeping a different index in the plaquette.
- Compare quark vs lepton plaquettes (if defined).
- Move on to Notebook 8 to see how prime structure feeds into xi and vacuum versus matter.
